In [ ]:
import pandas as pd

In [ ]:
import pydsm

In [ ]:
from pydsm import hydroh5

In [ ]:
hydro = hydroh5.HydroH5('d:/delta/dsm2_studies_master/studies/historical/output/hist_v2022_01.h5')

In [ ]:
hydro.get_channels()

In [ ]:
hydro.get_input_table('/hydro/input/channel')

In [ ]:
hydro.get_channel_avg_area(1)

In [ ]:
cids = list(hydro.get_channels().values.flatten())

In [ ]:
timewindow = '01JAN2014 - 01JAN2015'

In [ ]:
chan_areas = hydro.get_channel_avg_area(cids,timewindow)
chan_areas

In [ ]:
channels = hydro.get_input_table('/hydro/input/channel')

In [ ]:
chan_lengths = channels.set_index('chan_no')['length']
chan_lengths

In [ ]:
chan_vols = chan_lengths.values*chan_areas

In [ ]:
import hvplot.pandas

In [ ]:
# 1 acre-feet = 43559.9 cubic feet
# 1 maf (million acre-feet) = 43559.9*1e6
maf_factor = 43559.9*1e6

In [ ]:
chan_vols_maf = chan_vols/maf_factor

In [ ]:
chan_vols_maf.sum(axis=1).hvplot()

In [ ]:
hydro.get_reservoirs()

In [ ]:
list(hydro.get_reservoirs().values.flatten())

In [ ]:
reservoirs_height = hydro.get_reservoir_height(
    list(hydro.get_reservoirs().values.flatten()), timewindow)
reservoirs_height

In [ ]:
reservoirs = hydro.get_input_table('/hydro/input/reservoir')
reservoirs

In [ ]:
# reservoir areas are in millions of square feet
reservoirs_volume = (reservoirs.area.values*1e6)*reservoirs_height

In [ ]:
reservoirs_volume.sum(axis=1).hvplot()

In [ ]:
reservoirs_volume_maf = reservoirs_volume/maf_factor

In [ ]:
reservoirs_volume_maf.sum(axis=1).hvplot()

In [ ]:
dsm2_full_grid_vol = chan_vols_maf.sum(axis=1)+reservoirs_volume_maf.sum(axis=1)
dsm2_full_grid_vol.hvplot(title='DSM2 Volume (Full Extent)',ylabel='Volume (million acre-feet)')

In [ ]:
chans_suisun_marsh = pd.read_csv('suisun_marsh_channel_numbers.csv')

In [ ]:
chans_suisun_marsh.channel_nu.values

In [ ]:
suisun_marsh_cols = chan_vols_maf.columns[
    chan_vols_maf.columns.astype(int).isin(chans_suisun_marsh.channel_nu.values)]
not_suisun_marsh_cols = chan_vols_maf.columns[~
    chan_vols_maf.columns.astype(int).isin(chans_suisun_marsh.channel_nu.values)]


In [ ]:
chan_vols_maf[suisun_marsh_cols].sum(axis=1).hvplot(title='DSM2 Volume (Suisun Marsh)',ylabel='Volume (million acre-feet)')

In [ ]:
chan_vols_maf[not_suisun_marsh_cols].sum(axis=1).hvplot()

In [ ]:
(chan_vols_maf[not_suisun_marsh_cols].sum(
    axis=1)+reservoirs_volume_maf.sum(
    axis=1)).hvplot(title='DSM2 Volume (Delta only)',ylabel='Volume (million acre-feet)')